In [22]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import pytorch_lightning as pl

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
import matplotlib
from matplotlib.ticker import MaxNLocator

from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from torchmetrics.functional import accuracy

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [23]:
Concatnated = pd.read_csv('Concatnated.csv')    #.set_index('Time')
Concatnated = np.array(Concatnated[:292980], dtype=np.float32)
#Concatnated = torch.from_numpy(Concatnated)
print(Concatnated.shape)

(292980, 447)


In [24]:
CreatData = []
X = Concatnated[:, :446]
y = Concatnated[:,446]
strat_time = 0
timestamp = 10
length = len(Concatnated)

for i in range(int(abs(length/10-1))):
    CreatData.append((X[strat_time:timestamp, :446], y[timestamp]))

    strat_time = strat_time + 10
    timestamp = timestamp + 10

In [25]:
CreatData[0]

(array([[3.0301287e+00, 2.0545425e+01, 2.3036230e+01, ..., 4.0419906e-04,
         1.1364905e-04, 5.6682518e-03],
        [6.0662766e+00, 4.1093773e+01, 4.6072437e+01, ..., 8.1914989e-04,
         2.2856677e-04, 1.1336603e-02],
        [6.0843334e+00, 4.1102531e+01, 4.6072365e+01, ..., 8.5140375e-04,
         2.3237262e-04, 1.1336901e-02],
        ...,
        [6.3550611e+00, 4.1233990e+01, 4.6071308e+01, ..., 1.3349598e-03,
         2.8943276e-04, 1.1341364e-02],
        [6.4452510e+00, 4.1277821e+01, 4.6070957e+01, ..., 1.4960399e-03,
         3.0844129e-04, 1.1342849e-02],
        [6.5474334e+00, 4.1327507e+01, 4.6070553e+01, ..., 1.6785341e-03,
         3.2997734e-04, 1.1344530e-02]], dtype=float32),
 0.0)

In [26]:
train_dataset, test_dataset = train_test_split(CreatData, test_size=0.2)
print("Number of Training Sequences: ", len(train_dataset))
print("Number of Testing Sequences: ", len(test_dataset))

Number of Training Sequences:  23437
Number of Testing Sequences:  5860


In [27]:
batch_size = 64
num_epochs = 5
learning_rate = 0.001

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                         shuffle=False)

In [28]:
classes = ('Normal', 'Fuzz', 'DoS', 'Replay')

In [29]:
# get some random training images
dataiter = iter(train_loader)
sets, labels = dataiter.next()

print(sets[0].shape)
print(sets[0].ndim)

print(sets.shape)
print(sets.ndim)

torch.Size([10, 446])
2
torch.Size([64, 10, 446])
3


In [30]:
'''m = nn.Conv1d(16, 33, 3, stride=2)
input = torch.randn(20, 16, 50)
output = m(input)'''

'm = nn.Conv1d(16, 33, 3, stride=2)\ninput = torch.randn(20, 16, 50)\noutput = m(input)'

In [31]:
x = torch.randn(20, 16, 50)
x[0].shape

torch.Size([16, 50])

In [56]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv1d(10, 128, 5)
        self.pool = nn.MaxPool1d(2)
        self.conv2 = nn.Conv1d(128, 4, 5)
        
        self.fc1 = nn.Linear(4*108, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 4)

    def forward(self, x):
        # -> n, 3, 32, 32
        x = self.pool(F.relu(self.conv1(x)))  # -> n, 128, 442 -> n, 128, 221
        x = self.pool(F.relu(self.conv2(x)))  # -> n, 4, 217 -> n, 4, 108 
        x = x.view(-1, 4*108)            # -> n, 400
        x = F.relu(self.fc1(x))               # -> n, 120
        x = F.relu(self.fc2(x))               # -> n, 84
        x = self.fc3(x)                       # -> n, 10
        return x

In [57]:
model = ConvNet().to(device)
sets = sets.to(device)

In [58]:
output1 = model.conv1(sets)
output1.shape

torch.Size([13, 128, 442])

In [59]:
output2 = model.conv2(output1)
output2.shape

torch.Size([13, 4, 438])

In [60]:
p = model.pool(output2)
p.shape

torch.Size([13, 4, 219])

In [36]:
conv1 = nn.Conv1d(10, 128, 5)
print(conv1.weight.shape)

conv2 = nn.Conv1d(128, 4, 5)
print(conv2.weight.shape)

fc1 = nn.Linear(4, 120)
print(fc1.weight.shape)

torch.Size([128, 10, 5])
torch.Size([4, 128, 5])
torch.Size([120, 4])


In [37]:
model = ConvNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [43]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (sets, labels) in enumerate(train_loader):
        # origin shape: [4, 3, 32, 32] = 4, 3, 1024
        # input_layer: 3 input channels, 6 output channels, 5 kernel size
        #print(sets.shape)
        sets = sets.to(device)
        labels = labels.type(torch.LongTensor)
        labels = labels.to(device)
        

        # Forward pass
        outputs = model(sets)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

print('Finished Training')

Epoch [1/5], Step [100/367], Loss: nan
Epoch [1/5], Step [200/367], Loss: nan
Epoch [1/5], Step [300/367], Loss: nan
Epoch [2/5], Step [100/367], Loss: nan
Epoch [2/5], Step [200/367], Loss: nan
Epoch [2/5], Step [300/367], Loss: nan
Epoch [3/5], Step [100/367], Loss: nan
Epoch [3/5], Step [200/367], Loss: nan
Epoch [3/5], Step [300/367], Loss: nan
Epoch [4/5], Step [100/367], Loss: nan
Epoch [4/5], Step [200/367], Loss: nan
Epoch [4/5], Step [300/367], Loss: nan
Epoch [5/5], Step [100/367], Loss: nan
Epoch [5/5], Step [200/367], Loss: nan
Epoch [5/5], Step [300/367], Loss: nan
Finished Training


In [47]:
PATH = './cnn.pth'

In [48]:
torch.save(model.state_dict(), PATH)

In [41]:
len(train_loader)

367

In [51]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(4)]
    n_class_samples = [0 for i in range(4)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')

TypeError: only integer tensors of a single element can be converted to an index